# Optimal transport

*Selected Topics in Mathematical Optimization*

**Michiel Stock** ([email](michiel.stock@ugent.be))

![](Figures/logo.png)

In [ ]:
from optimal_transport import red, green, orange, yellow, blue, black
import matplotlib.pyplot as plt
import numpy as np
from optimal_transport import pairwise_distances
%matplotlib inline

## Cell tracking

In a microscopy imaging experiment we monitor ten moving cells at time $t_1$ and some time later at time $t_2$. Between these times, the cells have moved. An image processing algorithm determined the coordinates of every cell in the two images. We want to know which cell in the first corresponds to the second image. To this end, search the assignment that minimizes the sum of the squared Euclidian distances between cells from the first image versus the corresponding cell of the second image.

1. `X1` and `X2` contain the $x,y$ coordinates of the cells for the two images. Compute the matrix $C$ containing the pairwise squared Euclidean distance. You can use the function `pairwise_distances` from `sklearn`.
2. Complete the function `monge_brute_force` to use brute-force search for the best permutation.
3. Make a plot connecting the cells.

In [ ]:
from cell_tracking import X1, X2, plot_cells

In [ ]:
# all permutations can easily be generated in python
from itertools import permutations

for perm in permutations([1, 2, 3]):
    print(perm)

In [ ]:
fig, ax = plot_cells(X1, X2)

In [ ]:
def monge_brute_force(C):
    """
    Solves the Monge assignment problem using
    brute force.

    Inputs:
        - C: cost matrix (square, size n x n)

    Outputs:
        - best_perm: optimal assigments (list of n indices matching the rows
                to the columns)
        - best_cost: optimal cost corresponding to the best permutation

    DO NOT USE FOR PROBLEMS OF A SIZE LARGER THAN 12!!!
    """
    n, m = C.shape
    assert n==m  # C should be square
    best_perm = None
    best_cost = np.inf
    # loop over all permutations and to find the
    # matching with the lowest cost
    return best_perm, best_cost

In [ ]:
from optimal_transport import monge_brute_force

In [ ]:
# get the cost matrix (i.e. pairwise squared
# Euclidean distances between the cells at the different times)

C = ...

In [ ]:
# get matching

best_perm, best_cost = monge_brute_force(C)

In [ ]:
# make a plot with the connections of the cells

## Cell differentiation

Three types of cells are cultured together. At $t_1$ we know the expression of some cells of every type (two genes). After some time $t_2$, the cells have multiplied are have differentiated somewhat. A new gene expression analysis is done for a set of cells from the culture (without information about the type). How did the expression change for every type?

1. Link the cells from the two time points using OT. Use Sinkhorn with $\lambda=10$ and squared Euclidean distance for cost.
2. Plot the mapping (use the \texttt{alpha} argument to set the shade of a color).
3. Compute the `drift' (difference in average gene expression) in gene expression for every cell type. 

In [ ]:
# X1 and X2 are gene expressions for the cells at time 1 and 2
# y1 is the indicator of the type of cells, only known at t1
from cell_differentiation import X1, X2, y1, plot_cells

In [ ]:
fig, ax = plt.subplots()
plot_cells(ax)

In [ ]:
def compute_optimal_transport(C, a, b, lam, epsilon=1e-8,
                verbose=False, return_iterations=False):
    """
    Computes the optimal transport matrix and Slinkhorn distance using the
    Sinkhorn-Knopp algorithm

    Inputs:
        - C : cost matrix (n x m)
        - a : vector of marginals (n, )
        - b : vector of marginals (m, )
        - lam : strength of the entropic regularization
        - epsilon : convergence parameter
        - verbose : report number of steps while running
        - return_iterations : report number of iterations till convergence,
                default False

    Output:
        - P : optimal transport matrix (n x m)
        - dist : Sinkhorn distance
        - n_iterations : number of iterations, if `return_iterations` is set to
                        True
    """
    n, m = C.shape
    P = np.exp(- lam * C)
    iteration = 0
    while True:
        iteration += 1
        u = P.sum(1)  # marginals of rows
        max_deviation = np.max(np.abs(u - a))
        if verbose: print('Iteration {}: max deviation={}'.format(
                            iteration, max_deviation
                        ))
        if max_deviation < epsilon:
            break
        # scale rows
        ...
        # scale columns
        ...
    if return_iterations:
        return P, np.sum(P * C), iteration
    else:
        return P, np.sum(P * C)

In [ ]:
from optimal_transport import compute_optimal_transport

In [ ]:
# get the cost matrix (i.e. pairwise squared
# Euclidean distances of the expression vectors
# of the cells at the different times)

C = ...

In [ ]:
# get matching
P, _ = compute_optimal_transport(...

In [ ]:
# plot the cells with the mapping between the times

In [ ]:
# compute the drift (average change in gene expression
# for different classes between the two time points)

## Illustration: color transfer

This is a demonstration of a simple color transfer using optimal transport.

In [ ]:
from optimal_transport import compute_optimal_transport
from skimage import io
from sklearn.cluster import MiniBatchKMeans as KMeans
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import seaborn as sns
sns.set_style('white')

In [ ]:
# change as you see fit!
image_name1 = 'Figures/butterfly3.jpg'
image_name2 = 'Figures/butterfly2.jpg'

n_clusters = 400

In [ ]:
def clip_image(im):
    """
    Clips an image such that its values are between 0 and 255
    """
    return np.maximum(0, np.minimum(im, 255))

class Image():
    """simple class to work with an image"""
    def __init__(self, image_name, n_clusters=100, use_location=True):
        super(Image, self).__init__()
        self.image = io.imread(image_name) + 0.0
        self.shape = self.image.shape
        n, m, _ = self.shape
        X = self.image.reshape(-1, 3)
        if use_location:
            col_indices = np.repeat(np.arange(n), m).reshape(-1,1)
            row_indices = np.tile(np.arange(m), n).reshape(-1,1)
        #self.standardizer = StandardScaler()
        #self.standardizer.fit_transform(
            self.X = np.concatenate([X, row_indices, col_indices], axis=1)
        else: self.X = X
        self.kmeans = KMeans(n_clusters=n_clusters)
        self.kmeans.fit(self.X)

    def compute_clusted_image(self, center_colors=None):
        """
        Returns the image with the pixels changes by their cluster center

        If center_colors is provided, uses these for the clusters, otherwise use
        centers computed by K-means.
        """
        clusters = self.kmeans.predict(self.X)
        if center_colors is None:
            X_transformed = self.kmeans.cluster_centers_[clusters,:3]
        else:
            X_transformed = center_colors[clusters,:3]
        return clip_image(X_transformed).reshape(self.shape)

    def get_color_distribution(self):
        """
        Returns the distribution of the colored pixels

        Returns:
            - counts : number of pixels in each cluster
            - centers : colors of every cluster center
        """
        clusters = self.kmeans.predict(self.X)
        count_dict = Counter(clusters)
        counts = np.array([count_dict[i] for i in range(len(count_dict))],
                        dtype=float)
        centers = self.kmeans.cluster_centers_
        return counts, clip_image(centers[:,:3])

print('loading and clustering images...')
image1 = Image(image_name1, n_clusters=n_clusters)
image2 = Image(image_name2, n_clusters=n_clusters)

r, X1 = image1.get_color_distribution()
c, X2 = image2.get_color_distribution()


In [ ]:
print('loading and clustering images...')
image1 = Image(image_name1, n_clusters=n_clusters)
image2 = Image(image_name2, n_clusters=n_clusters)

r, X1 = image1.get_color_distribution()
c, X2 = image2.get_color_distribution()

C = pairwise_distances(X1, X2, metric="sqeuclidean")

In [ ]:
print('performing optimal transport...')
P, d = compute_optimal_transport(C, r/r.sum(), c/c.sum(), 1e-2)

sns.clustermap(P, row_colors=X1/255, col_colors=X2/255,
        yticklabels=[], xticklabels=[])
plt.savefig('Figures/color_mapping.png')

In [ ]:
print('computing and plotting color distributions...')
X1to2 = P.sum(1)**-1 * P @ X2
X2to1 = P.sum(0)**-1 * P.T @ X1

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axes[0, 0].imshow(image1.image/255)
axes[0, 1].imshow(image2.image/255)

axes[1, 0].imshow(image1.compute_clusted_image(X1to2)/255)
axes[1, 1].imshow(image2.compute_clusted_image(X2to1)/255)

for ax in axes.flatten():
    ax.set_xticks([])
    ax.set_yticks([])